# MVVM + Hexagonal in SEVA — Part 2 (View ↔ ViewModel ↔ UseCase)

## What we build in this notebook

We build one small Tkinter app that can:
- Start a demo experiment run
- Poll progress
- Show status text + progress bar

We keep the **polling example** exactly as before, but restructure it using **dependency injection**:
- View gets its dependencies (variables + callbacks) in `__init__`.
- ViewModel owns state + commands.
- UseCase runs the workflow and depends on `JobPort`.

As in **Part 1**, adapters were introduced there. Here we reuse **`JobRestMock`** to keep focus on MVVM.

In [ ]:
from pathlib import Path
import sys

# Allow notebook execution from docs/ while importing the project package.
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) View + Dependency Injection (DI)

### What we are programming in this step
We build a **Tkinter View** that only does UI concerns:
- Create widgets (labels, progressbar, buttons)
- Bind widgets to `StringVar` / `DoubleVar`
- Call *callbacks* when the user clicks buttons

### Why dependency injection here?
Instead of the View creating state or logic itself, we **inject** everything it needs:
- `status_text` (`tk.StringVar`) — what the label displays
- `progress_pct` (`tk.DoubleVar`) — what the progressbar displays
- `on_start`, `on_poll` — functions to call when buttons are clicked

This is the core idea:
> The View is **dumb UI**. It receives data + callbacks and never touches business logic.

**Data flow (conceptually):**
```
View  <-- reads -->  tk.Variable  <-- owned by -->  ViewModel
View  -- calls -->   callbacks     <-- methods on --> ViewModel
```


In [ ]:
import tkinter as tk
from tkinter import ttk
from dataclasses import dataclass


@dataclass
class StartVmState:
    """Pure UI state.

    This dataclass contains *no* Tkinter objects and *no* I/O.
    It is a simple, testable container that a ViewModel can mutate.
    """
    status_text: str = "Idle"
    progress_pct: float = 0.0
    is_running: bool = False
    start_clicks: int = 0


class StartViewClassic:
    """Classic View: dependencies injected in __init__ (no business logic)."""

    def __init__(self, root: tk.Tk, status_text: tk.StringVar, progress_pct: tk.DoubleVar, on_start, on_poll):
        """
        Classic View.

        Note what gets injected:
        - Tk variables (`status_text`, `progress_pct`) that widgets bind to
        - Callback functions (`on_start`, `on_poll`) that buttons call

        The View does *not* own state and does *not* implement behavior.
        """
        self.root = root
        self.root.title("MVVM Demo — Classic View")

        # Container frame for the UI
        outer = ttk.Frame(root, padding=12)
        outer.pack(fill="both", expand=True)

        # ---- READ-ONLY UI elements (they display injected Tk variables) ----

        ttk.Label(outer, text="Experiment status").pack(anchor="w")
        ttk.Label(outer, textvariable=status_text).pack(anchor="w", pady=(2, 6))
        ttk.Progressbar(outer, maximum=100, variable=progress_pct, length=300).pack(anchor="w", pady=(0, 8))

        row = ttk.Frame(outer)
        row.pack(anchor="w")
        ttk.Button(row, text="Start", command=on_start).pack(side="left", padx=(0, 6))
        ttk.Button(row, text="Poll", command=on_poll).pack(side="left")

In [ ]:
# Step 1 runnable demo: View only (no ViewModel yet)
#
# Run this cell after Step 1 to start the UI immediately.
# Close the window to continue to Step 2.

root = tk.Tk()

# Injected UI variables (normally owned by a ViewModel)
status_text = tk.StringVar(value="Idle (Step 1: View only)")
progress_pct = tk.DoubleVar(value=0.0)

# Injected callbacks (dummy behavior for Step 1)
def on_start():
    # Simulate a state change that would usually be triggered by a ViewModel
    status_text.set("Start clicked (dummy handler)")
    progress_pct.set(10.0)

def on_poll():
    # Simulate polling by incrementing the progress value
    progress_pct.set(min(100.0, progress_pct.get() + 7.5))

_ = StartViewClassic(root, status_text, progress_pct, on_start, on_poll)
root.mainloop()


## 2) ViewModel (state + commands)

### What we are programming in this step
Now we introduce a **ViewModel** that:
1. Owns a **pure Python state** object (`StartVmState`)
2. Owns the **Tk variables** (`StringVar`, `DoubleVar`) that the View binds to
3. Exposes **command methods** that the View can call (e.g. when a button is clicked)

### Programming idea: "Commands + Sync"
A common beginner-friendly pattern is:
- **Command method** (triggered by the UI) updates the Python `state`
- Then a small `_sync()` method copies the state into Tk variables

That keeps responsibilities clear:
- State changes happen in one place (ViewModel)
- UI updates happen through bindings (Tk variables)

### Very important: how the View calls the ViewModel
When we construct the View, we **inject ViewModel methods** into it:

- `on_start = vm.on_start_clicked`
- `on_poll  = vm.on_poll_clicked`

This is what “wiring” means: the View knows *only* functions, not business logic.


In [ ]:
class StartViewModelStepB:
    """ViewModel-only step (no use case yet)."""

    def __init__(self):
        self.state = StartVmState()
        # These variables are injected into the View.
        self.status_text = tk.StringVar(value=self.state.status_text)
        self.progress_pct = tk.DoubleVar(value=self.state.progress_pct)

    def can_start(self) -> bool:
        return not self.state.is_running

    def _sync(self):
        """Copy Python state → Tk variables.

        Widgets are bound to Tk variables, so updating the variables updates the UI.
        """
        self.status_text.set(self.state.status_text)
        self.progress_pct.set(self.state.progress_pct)

    def on_start_clicked(self):
        """UI command: Start.

        This method is intended to be injected into the View as a callback.
        It performs *presentation logic* only (no I/O):
        - update state
        - sync state into Tk variables
        """
        if not self.can_start():
            self.state.status_text = "Already running"
            self._sync()
            return

        self.state.is_running = True
        self.state.start_clicks += 1
        self.state.progress_pct = 0.0
        self.state.status_text = f"Run #{self.state.start_clicks} started"
        self._sync()

    def on_poll_clicked(self):
        """UI command: Poll progress (Step 2 simulation).

        In this step we still don't have a UseCase. We simulate progress
        by incrementing the percentage in the ViewModel state.
        """
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            self._sync()
            return

        self.state.progress_pct = min(100.0, self.state.progress_pct + 10.0)
        self.state.status_text = f"Running... {self.state.progress_pct:.0f}%"

        if self.state.progress_pct >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"

        self._sync()

In [ ]:
# Step 2 assembled app: View + ViewModel + dependency injection
#
# Key idea: the View receives *methods of the ViewModel* as callbacks.
# That is how button clicks in the View trigger ViewModel logic.

root = tk.Tk()

vm_b = StartViewModelStepB()

# ---- WIRING (inject ViewModel into View) ----
# - The View binds to vm_b.status_text / vm_b.progress_pct
# - The View buttons call vm_b.on_start_clicked / vm_b.on_poll_clicked
view = StartViewClassic(
    root,
    vm_b.status_text,
    vm_b.progress_pct,
    vm_b.on_start_clicked,  # injected ViewModel method
    vm_b.on_poll_clicked,   # injected ViewModel method
)

root.mainloop()


### Step 2 recap

You just programmed a **ViewModel** with:
- **state** (`StartVmState`) as plain Python data
- **Tk variables** (`StringVar`, `DoubleVar`) as bindings for the View
- **command methods** (`on_start_clicked`, `on_poll_clicked`) that the View calls

Most importantly, the View was *wired* by injecting **ViewModel methods** as callbacks.


## 3) UseCase (port-based) + ViewModel orchestration

### Why we introduce a UseCase
Starting a job and polling its progress is **application/business workflow**.
If we keep that workflow inside the ViewModel, the ViewModel grows into a “god object”.

Instead, we create a **UseCase** that:
- Defines *what the app does* (start + poll)
- Depends on an abstract **Port** (`JobPort`)
- Can be tested independently of the UI

### Hexagonal architecture in one minute
- **Port**: an interface describing what we need (e.g. “start batch”, “poll batch”)
- **Adapter**: a concrete implementation (here: `JobRestMock` so we stay offline)
- **UseCase**: orchestrates calls to the port and returns simple values

Conceptual flow:
```
View  →  ViewModel  →  UseCase  →  JobPort  →  Adapter (Mock/REST/DB)
```

### What changes in the ViewModel now?
In Step 2 the ViewModel faked progress locally.
In Step 3 the ViewModel becomes an orchestrator:
- UI event → call UseCase
- Take UseCase results → update UI state / Tk variables


In [ ]:
# Ports and adapters:
# - JobPort is the abstract interface (the "port")
# - JobRestMock is a concrete implementation (an "adapter") used for this offline tutorial
from seva.adapters.job_rest_mock import JobRestMock
from seva.domain.ports import JobPort


class StartAndPollProgressUseCase:
    """UseCase: start a group and poll progress through JobPort.

    What this UseCase does:
    - `start()` prepares a plan and asks the port to start a batch
    - `poll_progress_pct()` asks the port for a snapshot and computes progress in percent

    The UseCase contains *workflow logic*, not UI code.
    """

    def __init__(self, job_port: JobPort):
        self.job_port = job_port
        self.active_group_id = None

    def start(self) -> str:
        # The plan describes what should run. In a real app this might come from user input.
        plan = JobRestMock.example_plan()
        # Delegate the actual start to the port (REST/mock/etc.)
        group_id, _runs = self.job_port.start_batch(plan)
        self.active_group_id = group_id
        return group_id

    def poll_progress_pct(self) -> float:
        # Ask the port for the latest snapshot and map it to a simple percentage for the UI.
        if not self.active_group_id:
            return 0.0

        snapshot = self.job_port.poll_group(self.active_group_id)

        # Simplified extraction for tutorial readability.
        for box in snapshot.get("boxes", {}).values():
            runs = box.get("runs", [])
            if runs:
                return float(runs[0].get("progress_pct", 0.0))
        return 0.0

In [ ]:
class StartViewModel:
    """ViewModel: orchestrates use-case calls + maps results to UI state."""

    def __init__(self, use_case: StartAndPollProgressUseCase):
        self.use_case = use_case  # injected dependency (UseCase)
        self.state = StartVmState()
        self.status_text = tk.StringVar(value=self.state.status_text)
        self.progress_pct = tk.DoubleVar(value=self.state.progress_pct)

    def can_start(self) -> bool:
        return not self.state.is_running

    def _sync(self):
        """Copy ViewModel state → Tk variables (updates the UI via bindings)."""
        self.status_text.set(self.state.status_text)
        self.progress_pct.set(self.state.progress_pct)

    def on_start_clicked(self):
        """UI command: Start.

        What happens programmatically:
        1) Guard: check whether starting is allowed
        2) Call the UseCase to start the workflow
        3) Update UI state and sync into Tk variables
        """
        if not self.can_start():
            self.state.status_text = "Already running"
            self._sync()
            return

        group_id = self.use_case.start()
        self.state.start_clicks += 1
        self.state.is_running = True
        self.state.progress_pct = 0.0
        self.state.status_text = f"Started group: {group_id}"
        self._sync()

    def on_poll_clicked(self):
        """UI command: Poll.

        What happens programmatically:
        1) Call the UseCase to obtain the latest progress percentage
        2) Map that value into UI state (text + progress)
        3) Sync into Tk variables so the View updates
        """
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            self._sync()
            return

        progress = self.use_case.poll_progress_pct()
        self.state.progress_pct = progress

        if progress >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"
        else:
            self.state.status_text = f"Running... {progress:.1f}%"

        self._sync()

In [ ]:
# Step 3 assembled app: View + ViewModel + UseCase + scheduler
#
# In this step, polling is implemented with a small scheduler loop using `root.after`.
# We keep scheduling in this runner cell (composition root), so the ViewModel stays easy to test.

root = tk.Tk()

# --- Dependency graph (composition) ---
# Adapter (JobRestMock) implements JobPort
# UseCase depends on JobPort
# ViewModel depends on UseCase
# View depends on Tk variables + callbacks (injected from the ViewModel)
uc = StartAndPollProgressUseCase(JobRestMock())
vm = StartViewModel(uc)
view = StartViewClassic(root, vm.status_text, vm.progress_pct, vm.on_start_clicked, vm.on_poll_clicked)


def _tick_poll():
    """Polling loop.

    This function is scheduled repeatedly by Tkinter.
    If the ViewModel is in 'running' state, we trigger its poll command.
    """
    if vm.state.is_running:
        vm.on_poll_clicked()

    # Schedule the next tick in 1000 ms (1 second)
    root.after(1000, _tick_poll)


_tick_poll()
root.mainloop()


### Step 3 recap

Now the workflow lives in a **UseCase** (start + poll) which depends on an abstract **Port** (`JobPort`).
The ViewModel's job becomes orchestration:
- translate UI events → UseCase calls
- map UseCase results → UI state/bindings


## 4) Swap to a visually distinct (more modern) View

### What we are practicing here
A good MVVM setup lets you **swap the View** (layout/styling) without rewriting:
- the UseCase (workflow logic)
- the ViewModel (state + commands)
- the dependency injection wiring

### Programmatically, what stays the same?
- The View constructor still receives:
  - the same Tk variables (`status_text`, `progress_pct`)
  - the same callbacks (`on_start`, `on_poll`)
- The ViewModel + UseCase are identical to Step 3

So the only change is: `StartViewClassic(...)` → `StartViewModern(...)`.
That is a strong signal that responsibilities are well separated.


In [ ]:
class StartViewModern:
    """More visually distinct view (layout/styling only).

    Important: the signature is the same as the classic view.
    That means we can swap views without touching ViewModel / UseCase.
    """

    def __init__(self, root: tk.Tk, status_text: tk.StringVar, progress_pct: tk.DoubleVar, on_start, on_poll):
        self.root = root
        self.root.title("MVVM Demo — Modern View")

        # Optional styling tweaks (purely cosmetic)
        style = ttk.Style(root)
        if "clam" in style.theme_names():
            style.theme_use("clam")

        style.configure("Card.TFrame", padding=14)
        style.configure("Title.TLabel", font=("TkDefaultFont", 12, "bold"))

        shell = ttk.Frame(root, padding=16)
        shell.pack(fill="both", expand=True)

        card = ttk.Frame(shell, style="Card.TFrame")
        card.pack(fill="x")

        ttk.Label(card, text="Experiment Runner", style="Title.TLabel").pack(anchor="w")
        ttk.Label(card, text="Status").pack(anchor="w", pady=(8, 0))
        ttk.Label(card, textvariable=status_text).pack(anchor="w", pady=(0, 8))
        ttk.Progressbar(card, maximum=100, variable=progress_pct, length=320).pack(anchor="w", pady=(0, 10))

        action_row = ttk.Frame(card)
        action_row.pack(anchor="w")
        ttk.Button(action_row, text="▶ Start", command=on_start).pack(side="left", padx=(0, 8))
        ttk.Button(action_row, text="⟳ Poll", command=on_poll).pack(side="left")

In [ ]:
# Step 4 assembled app: same VM/UseCase, modern View (DI in constructor)
root = tk.Tk()

# Same composition as Step 3
uc_m = StartAndPollProgressUseCase(JobRestMock())
vm_m = StartViewModel(uc_m)

# Only change: different View class, same injected dependencies
modern_view = StartViewModern(
    root,
    vm_m.status_text,
    vm_m.progress_pct,
    vm_m.on_start_clicked,
    vm_m.on_poll_clicked,
)


def _tick_poll_modern():
    """Same scheduler loop as Step 3, just with a different View."""
    if vm_m.state.is_running:
        vm_m.on_poll_clicked()
    root.after(1000, _tick_poll_modern)


_tick_poll_modern()
root.mainloop()
